In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:

from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"lorenzodelmedico","key":"2e8bbd994a00cda8a968355dd7bc542e"}'}

In [8]:
!ls -lha kaggle.json

-rw-r--r-- 1 root root 72 Jun  8 08:32 kaggle.json


In [9]:

!pip install -q kaggle
!mkdir -p ~/.kaggle #Create the directory

In [10]:
!cp kaggle.json ~/.kaggle/

!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d rqfvitor/95-images-of-each-of-1281-pokemon

!unzip 95-images-of-each-of-1281-pokemon.zip

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: datasets/train/wo-chien/a4ec5466-728e-47bb-8269-a7e886e2de2d.png  
  inflating: datasets/train/wo-chien/a78de547-beb8-4557-8f76-57cd89e95257.png  
  inflating: datasets/train/wo-chien/a972ddf3-9b79-4382-abce-ca2c9b8751e7.png  
  inflating: datasets/train/wo-chien/a9f23850-f24f-4693-adb8-e371d1938f69.png  
  inflating: datasets/train/wo-chien/ab8364b9-e46e-460d-96dc-e4ddb9606052.png  
  inflating: datasets/train/wo-chien/b03aca44-b629-49a1-9d73-23b2a9c4bbe5.png  
  inflating: datasets/train/wo-chien/b31a0a6a-80f6-48a7-b65d-2c6e737ab3d4.png  
  inflating: datasets/train/wo-chien/b3646835-190f-4783-9297-b34a8d49306b.png  
  inflating: datasets/train/wo-chien/b5965603-1dbf-4e53-ba0a-f1488a7c8708.png  
  inflating: datasets/train/wo-chien/b823878f-da1f-4b87-80ee-7fca3385cbdb.png  
  inflating: datasets/train/wo-chien/ba4b1c3e-26a1-4281-8db3-ac982841a605.png  
  inflating: datasets/train/wo-chien/bd564

In [2]:
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from PIL import Image
import numpy as np

In [3]:
!ls

95-images-of-each-of-1281-pokemon.zip  drive	    sample_data
datasets			       kaggle.json


In [4]:
# Function to process a single image
def process_image(img_path):
    label = os.path.basename(os.path.dirname(img_path))
    
    img = Image.open(img_path)
    img = img.convert("RGB")
    flattened_img = np.array(img).reshape(-1)
    
    return flattened_img, label

In [5]:
data_dir = "./datasets/train/"

In [6]:
#Initialize empty lists to store the image vectors and their corresponding labels:
image_vectors = []
labels = []

In [7]:
total_files = sum([len(files) for _, _, files in os.walk(data_dir)])
total_dir = sum([len(dir) for _, dir, files in os.walk(data_dir)])

In [11]:
print(total_dir)

1281


In [8]:
with ThreadPoolExecutor() as executor:
    futures = []
    for root, dirs, files in tqdm(os.walk(data_dir), desc="Processing images"):
        for file in files:
            img_path = os.path.join(root, file)
            futures.append(executor.submit(process_image, img_path))

    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing progress"):
        flattened_img, label = future.result()
        image_vectors.append(flattened_img)
        labels.append(label)


Processing images: 1282it [01:13, 17.50it/s]
Processing progress: 100%|██████████| 132202/132202 [02:27<00:00, 896.37it/s]


In [15]:
len(image_vectors)

132202

In [9]:
max_chunk_size = 10000

In [10]:
# Split the array into smaller chunks
chunks = [image_vectors[i:i+max_chunk_size] for i in range(0, len(image_vectors), max_chunk_size)]

In [11]:
# Print the number of chunks created
print("Total chunks:", len(chunks))

Total chunks: 14


In [12]:
import matplotlib.pyplot as plt
import tensorflow as tf

In [13]:
def pca_compress_decompress(X, n_components=30):

    # Convert the list of arrays to a single NumPy array
    X_concatenated = np.concatenate(X)

    # Convert data to float and scale it between 0 and 1
    X_concatenated = X_concatenated.astype(float) / 255.0

    # Calculate the mean vector
    mean_vector = np.mean(X_concatenated, axis=0)

    # Subtract the mean vector from the data
    X_centered = X_concatenated - mean_vector

    # calculating the covariance matrix of the mean-centered data.
    cov_mat = np.cov(X_centered , rowvar = False)

    #Calculating Eigenvalues and Eigenvectors of the covariance matrix
    eigen_values , eigen_vectors = np.linalg.eigh(cov_mat)

    #sort the eigenvalues in descending order
    sorted_index = np.argsort(eigen_values)[::-1]
    
    sorted_eigenvalue = eigen_values[sorted_index]
    #similarly sort the eigenvectors 
    sorted_eigenvectors = eigen_vectors[:,sorted_index]


    # select the first n eigenvectors, n is desired dimension
    # of our final reduced data.
    eigenvector_subset = sorted_eigenvectors[:,0:n_components]

    #Transform the data 
    X_reduced = np.dot(eigenvector_subset.transpose(),X_centered.transpose()).transpose()

    # Reconstruct the data from the projected space
    X_reconstructed = np.dot(X_reduced, eigenvector_subset.T) + mean_vector


    return X_reconstructed

In [ ]:
n_components = 500
X_reconstructed = pca_compress_decompress(chunks[0], n_components=n_components)

In [ ]:
# Split the reconstructed data back into individual arrays
reconstructed_list = np.split(X_reconstructed, np.cumsum([len(x) for x in chunks[0]])[:-1])